In [1]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms

In [2]:
data_dir = './data/train'

In [3]:
train_transforms = transforms.Compose([transforms.Resize(32),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.RandomRotation(30),
                                      transforms.RandomRotation(60),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])
data = datasets.ImageFolder(data_dir, transform=train_transforms)
train_data, val_data = torch.utils.data.random_split(data, [4000, 537])
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=True)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Layers
        self.conv1 = nn.Conv2d(3, 32, 5, 1)
        self.conv2 = nn.Conv2d(32, 64, 5, 1)
        self.fc1 = nn.Linear(64*5*5, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        # define forward propagation here
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2, 2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 64*5*5)
        x = self.fc1(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [5]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [6]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [7]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [8]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 30

for epoch in range(epochs-20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, val_loader)
    
optimizer = optim.Adam(model.parameters(), lr=0.0005)
for epoch in range(epochs-20, epochs-10):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, val_loader)     
    
optimizer = optim.Adam(model.parameters(), lr=0.0001)
for epoch in range(epochs-10, epochs):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, val_loader)

torch.save(model.state_dict(),"hw2_final1.pth")

Train Epoch: 1 [0/4000 (0%)]	Loss: 2.332090
Train Epoch: 1 [640/4000 (16%)]	Loss: 1.240669
Train Epoch: 1 [1280/4000 (32%)]	Loss: 1.413548
Train Epoch: 1 [1920/4000 (48%)]	Loss: 1.291968
Train Epoch: 1 [2560/4000 (63%)]	Loss: 1.072481
Train Epoch: 1 [3200/4000 (79%)]	Loss: 1.028726
Train Epoch: 1 [3840/4000 (95%)]	Loss: 0.746496

Test set: Average loss: 0.9117, Accuracy: 357/537 (66%)

Train Epoch: 2 [0/4000 (0%)]	Loss: 0.901478
Train Epoch: 2 [640/4000 (16%)]	Loss: 0.702296
Train Epoch: 2 [1280/4000 (32%)]	Loss: 0.564367
Train Epoch: 2 [1920/4000 (48%)]	Loss: 0.950831
Train Epoch: 2 [2560/4000 (63%)]	Loss: 0.980025
Train Epoch: 2 [3200/4000 (79%)]	Loss: 0.724869
Train Epoch: 2 [3840/4000 (95%)]	Loss: 0.446139

Test set: Average loss: 0.7839, Accuracy: 376/537 (70%)

Train Epoch: 3 [0/4000 (0%)]	Loss: 0.717299
Train Epoch: 3 [640/4000 (16%)]	Loss: 0.962902
Train Epoch: 3 [1280/4000 (32%)]	Loss: 0.737634
Train Epoch: 3 [1920/4000 (48%)]	Loss: 0.866351
Train Epoch: 3 [2560/4000 (63%)]	Lo


Test set: Average loss: 0.5946, Accuracy: 411/537 (77%)

Train Epoch: 22 [0/4000 (0%)]	Loss: 0.606145
Train Epoch: 22 [640/4000 (16%)]	Loss: 0.590877
Train Epoch: 22 [1280/4000 (32%)]	Loss: 0.437505
Train Epoch: 22 [1920/4000 (48%)]	Loss: 0.506164
Train Epoch: 22 [2560/4000 (63%)]	Loss: 0.401581
Train Epoch: 22 [3200/4000 (79%)]	Loss: 0.419345
Train Epoch: 22 [3840/4000 (95%)]	Loss: 0.475409

Test set: Average loss: 0.5825, Accuracy: 418/537 (78%)

Train Epoch: 23 [0/4000 (0%)]	Loss: 0.387221
Train Epoch: 23 [640/4000 (16%)]	Loss: 0.601162
Train Epoch: 23 [1280/4000 (32%)]	Loss: 0.623928
Train Epoch: 23 [1920/4000 (48%)]	Loss: 0.655793
Train Epoch: 23 [2560/4000 (63%)]	Loss: 0.455994
Train Epoch: 23 [3200/4000 (79%)]	Loss: 0.513008
Train Epoch: 23 [3840/4000 (95%)]	Loss: 0.371761

Test set: Average loss: 0.6125, Accuracy: 411/537 (77%)

Train Epoch: 24 [0/4000 (0%)]	Loss: 0.531532
Train Epoch: 24 [640/4000 (16%)]	Loss: 0.530038
Train Epoch: 24 [1280/4000 (32%)]	Loss: 0.634042
Train Ep